In [23]:
from config import db_config
import pandas as pd
import numpy as np
import bokeh
import pymysql
import sys
import datetime
import calendar

from bokeh.models import Circle, ColumnDataSource, Line, LinearAxis, Range1d, Legend
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.core.properties import value
from bokeh.palettes import Category20
from bokeh.models import Toggle, BoxAnnotation

output_notebook()  # output bokeh plots to jupyter notebook
output_file('2019 Jan 1 to May 6.html')

try:
    # print ("SQL CONNECTION")
    conn = pymysql.connect(host=db_config[0], user=db_config[1], passwd=db_config[2], db=db_config[3],
                           connect_timeout=10)
except Exception as e:
    print("Unable to connect to db")
    sys.exit()


sql = """
   SELECT
        aq.label, 
        CONVERT_TZ(from_unixtime(recorded_at),'+00:00','-05:00') as recorded_at, 
        temp_f,
        humidity, 
        pressure/10 as pressure,
        AQI
    FROM
        air_monitors mon
    JOIN
        air_quality aq
    ON
        mon.id = aq.id
        AND mon.status IS NOT NULL
    WHERE
        aq.recorded_at > UNIX_TIMESTAMP("2019-09-01")
"""

df = pd.read_sql(sql, conn)
monitors = df.label.unique()

Loading BokehJS ...

In [24]:
p = figure(
    title = 'AQI',
    x_axis_label ='Time',
    x_axis_type='datetime',
    y_range = (0,200),
    plot_width=900,
    plot_height=600
)

green_box =  BoxAnnotation(top=50, bottom=0, fill_color='green', fill_alpha=0.1)
yellow_box = BoxAnnotation(top=100, bottom=50, fill_color='yellow', fill_alpha=0.1)
orange_box = BoxAnnotation(top=150, bottom=100, fill_color='orange', fill_alpha=0.1)
red_box = BoxAnnotation(top=200, bottom=150, fill_color='red', fill_alpha=0.1)
purple_box = BoxAnnotation(top=300, bottom=200, fill_color='purple', fill_alpha=0.1)

p.add_layout(green_box)
p.add_layout(yellow_box)
p.add_layout(orange_box)
p.add_layout(red_box)
p.add_layout(purple_box)

In [25]:
# FOR PLOTTING MULTIPLE MONITORS OVER TIME
count = 0
palette_list = len(monitors) + 5
legend_list = []

for m in monitors:
    data = df[df.label == m]
    d = p.line(data.recorded_at, data.AQI, legend=m, line_width=2, color=Category20[palette_list][count])
    count +=1 

In [26]:
show(p)

In [ ]:
df.corr()

In [ ]:
df.recorded_at

In [ ]:
bd = df.groupby(['weekday','time'])['weekday','AQI'].mean()

In [27]:
import hurst

In [28]:
s = df[df.label == 'McKinley Park South'].AQI

In [31]:
H, c, data = hurst.compute_Hc(s, kind='change', simplified=True)

In [32]:
H

0.6180319159015656